In [12]:
import os
import re

from collections import defaultdict, Counter
import numpy as np

In [2]:
def Input(day):
    # Read an input
    filename = 'data/{}.txt'.format(day)
    return open(filename)

def Lines(day):
    # Read the input and convert to an array of strings.
    return Input(day).read().splitlines()

def vector(day):
    # Read the input, and convert it to a vector of numbers
    return list(map(float, Lines(day)))

## Day 1

In [3]:
# Read input
inp = vector(1)
           
# The answer to the first one is just the sum
print(sum(inp))

533.0


In [4]:
# For the second part, use a set to determine if we've seen 
# a frequency before
v = set()
ii = 0
s = 0
found = False
while not found:
    s = s + inp[ii % len(inp)]
    if s in v:
        print("Found freq: {} at iteration {} ({} passes)".format(
                s, ii, ii % len(inp)))
        break
    v.add(s)
    ii = ii + 1

Found freq: 73272.0 at iteration 130408 (528 passes)


## Day 2

In [5]:
def checksum(s):
    cnt = Counter()
    for l in s:
        cnt[l] += 1
    return (2 in cnt.values(), 3 in cnt.values())

assert(checksum('abcdef') == (False, False))
assert(checksum('bababc') == (True, True))
assert(checksum('abbcde') == (True, False))
assert(checksum('abcccd') == (False, True))
assert(checksum('aabcdd') == (True, False))
assert(checksum('abcdee') == (True, False))
assert(checksum('ababab') == (False, True))

inp = Lines(2)

twos = 0
threes = 0

for line in inp:
    cnt = checksum(line)
    if cnt[0]:
        twos += 1
    if cnt[1]:
        threes += 1
        
print('Twos: {}, Threes: {}, Checksum: {}'.format(twos, threes, twos * threes))     

Twos: 245, Threes: 36, Checksum: 8820


In [6]:
def diff(s1, s2):
    count = 0
    for (c1, c2) in zip(s1, s2):
        if c1 != c2:
            count += 1
    return count
    
fline1 = None
fline2 = None
    
for line1 in inp:
    for line2 in inp:
        if diff(line1, line2) == 1:
            fline1 = line1
            fline2 = line2
            break
    if fline1:
        break
        
print(fline1)
print(fline2)

out = ''
for (c1, c2) in zip(fline1, fline2):
    if c1 == c2:
        out += c1
print(out)

bpacnmglhizqygfsjixtlkwudr
bpacnmglhizqygfsjixtukwudr
bpacnmglhizqygfsjixtkwudr


## Day 3

In [7]:
def parse_claim(claim_str):
    ids, rem = claim_str.split('@')
    ids = int(ids.strip('#'))
    coords, size = rem.split(':')
    left, top = map(int, coords.split(','))
    width, height = map(int, size.split('x'))
    return (ids, left, top, width, height)


claims = []
for inp in Lines(3):
    claims.append(parse_claim(inp))
    
fabric = np.zeros((1000, 1000), dtype=np.int32)
for claim in claims:
    ids, left, top, width, height = claim
    fabric[left:left+width, top:top+height] +=  1
print(np.sum(fabric > 1))

119551


In [8]:
for claim in claims:
    ids, left, top, width, height = claim
    section = fabric[left:left+width, top:top+height]
    if np.all(section == 1):
        print(ids)

1124


## Day 4

In [195]:
inp = Lines(4)
r = re.compile('\[(\d+)-(\d+)-(\d+) (\d+):(\d+)\] (.*)')

logs = []
for line in inp:
    m = r.match(line)
    logs.append(m.groups())
logs = sorted(logs, key = lambda log: (log[0], log[1], log[2], log[3], log[4]))

guards = {}

last_guard = -1

for log in logs:
    match = re.search('Guard #(\d*)', log[5])
    if match:
        guard = int(match.group(1))
        last_guard = guard
        if guard not in guards:
            guards[guard] = {'asleep': [], 'awake': []}
    if log[5].find('asleep') >= 0:
        guards[last_guard]['asleep'].append((int(log[3]), int(log[4])))
    if log[5].find('wakes') >= 0:
        guards[last_guard]['awake'].append((int(log[3]), int(log[4])))
        
print(guards.keys())

def sub(awake, asleep):
    minutes = awake[1] - asleep[1]
    hours = awake[0] - asleep[0]
    if minutes >= 60:
        hours += 1
        minutes -= 60
    return (hours, minutes)

def add(t1, t2):
    minutes = t1[1] + t2[1]
    hours = t1[0] + t2[0]
    
    if minutes >= 60:
        hours += 1
        minutes -= 60
    return (hours, minutes)

for guard, values in guards.items():
    total = (0, 0)
    for asleep, awake in zip(values['asleep'], values['awake']):
        diff = sub(awake, asleep)
        total = add(total, diff)
    guards[guard]['total'] = total
    
guard_ids = sorted(guards, key=lambda g: guards[g]['total'], reverse=True)
print(guard_ids[0], guards[guard_ids[0]]['total'])

sleepiest = guards[guard_ids[0]]

minutes = np.zeros((1, 60) )
for asleep, awake in zip(sleepiest['asleep'], sleepiest['awake']):
    minutes[0, asleep[1]:awake[1]] += 1
    
sleepiest_minute = np.argmax(minutes)

print(guard_ids[0], sleepiest_minute, guard_ids[0] * sleepiest_minute)

dict_keys([887, 2441, 571, 1889, 2207, 971, 2963, 3539, 613, 1291, 239, 1021, 673, 1249, 911, 1877, 881, 1607, 331, 929, 1913, 79, 877])
2441 (8, 8)
2441 39 95199


In [186]:
array = np.zeros((len(guard_ids), 60), dtype=np.int32)

for ii, guard in enumerate(guard_ids):
    for asleep, awake in zip(guards[guard]['asleep'], guards[guard]['awake']):
        array[ii, int(asleep[1]): int(awake[1])] += 1
m = np.max(array)
x, y = np.where(array == m)
int(guard_ids[x[0]]) * y[0]

7887

7887